In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Shuffling and writing examples to /Users/mac/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteW60749/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/mac/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteW60749/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/mac/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteW60749/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/mac/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [5]:
encoder = info.features['text'].encoder

In [7]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [8]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [9]:
assert original_string == sample_string

In [10]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [11]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [12]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [15]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 650s 2s/step - loss: 0.6449 - accuracy: 0.5583 - val_loss: 0.5470 - val_accuracy: 0.7760
Epoch 2/10
391/391 [==============================] - 617s 2s/step - loss: 0.3462 - accuracy: 0.8497 - val_loss: 0.3417 - val_accuracy: 0.8578
Epoch 3/10
391/391 [==============================] - 573s 1s/step - loss: 0.2517 - accuracy: 0.9012 - val_loss: 0.3508 - val_accuracy: 0.8615
Epoch 4/10
391/391 [==============================] - 570s 1s/step - loss: 0.2120 - accuracy: 0.9213 - val_loss: 0.3319 - val_accuracy: 0.8729
Epoch 5/10
391/391 [==============================] - 556s 1s/step - loss: 0.1823 - accuracy: 0.9344 - val_loss: 0.3378 - val_accuracy: 0.8687
Epoch 6/10
391/391 [==============================] - 544s 1s/step - loss: 0.1600 - accuracy: 0.9437 - val_loss: 0.3775 - val_accuracy: 0.8693
Epoch 7/10
391/391 [==============================] - 521s 1s/step - loss: 0.1433 - accuracy: 0.9499 - val_loss: 0.4018 - val_accuracy: 0.8677